<h1>NEWS HEADLINE CLASSIFICATION</h1>  

<p>In this notebook we will try to classify news using <b>MULTINOMIAL NAIVE BAYES</b>. We will convert text into vector by using <b>Bag Of Words</b> model. We will use two different datasets from two differrent sources and merge them.Our objective is to find maximum accuracy possible</p> 

In [1]:
##Loading all the required libraries
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
import time

#library for regular expretion
import re

In [2]:
#Loading the data which is from Kaggle
#This dataset contains four news class -b for business, e- entertainment, -t for technology and science, -m for medical
data = pd.read_csv("uci-news-aggregator.csv",encoding='latin-1')

In [3]:
#Loading another data which contain 5 different class of news
#Nature_Environment , Politics, Lifestyle, Business_finance and Health
data1 = pd.read_csv("ArticlesNewsSitesData_2382.csv",sep=';')
data1.columns = ['TITLE','CATEGORY']
data1.head()

,TITLE,CATEGORY
0,http://domesticfuel.com/2014/05/02/oregon-inst...,Nature_Environment
1,http://peakoil.com/generalideas/peak-oil-theor...,Nature_Environment
2,http://theenergycollective.com/maxbaumhefner/3...,Nature_Environment
3,http://dailykos.com/story/2014/05/02/1296329/-...,Nature_Environment
4,http://theenergycollective.com/todayinenergy/3...,Nature_Environment


In [4]:
#Extracting news haedline from urls 
for i in range(2382):
    data1.TITLE[i] = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))', '', data1.TITLE[i])
    data1.TITLE[i] = re.sub(r'(\/[0-9]*\/)*[0-9]*\/', '', data1.TITLE[i])
    data1.TITLE[i]=data1.TITLE[i].replace("-"," ")
    



In [5]:
data1.tail()

,TITLE,CATEGORY
2377,The Who s Roger Daltry Rocking for Philly Can...,Lifestyle_leisure
2378,,Lifestyle_leisure
2379,stylehoroscope5e1a15f0 98e5 11e3 80ac 63a8ba7f...,Lifestyle_leisure
2380,dario franchitti interview indianapolis 500 ca...,Lifestyle_leisure
2381,flower power oils for loss and the stages of g...,Lifestyle_leisure


In [6]:
data1['TITLE'].replace('', np.nan, inplace=True)
data1.tail()

,TITLE,CATEGORY
2377,The Who s Roger Daltry Rocking for Philly Can...,Lifestyle_leisure
2378,NaN,Lifestyle_leisure
2379,stylehoroscope5e1a15f0 98e5 11e3 80ac 63a8ba7f...,Lifestyle_leisure
2380,dario franchitti interview indianapolis 500 ca...,Lifestyle_leisure
2381,flower power oils for loss and the stages of g...,Lifestyle_leisure


In [7]:
data1.dropna(subset=['TITLE'], inplace=True)
data1.tail()

,TITLE,CATEGORY
2376,people watching.html,Lifestyle_leisure
2377,The Who s Roger Daltry Rocking for Philly Can...,Lifestyle_leisure
2379,stylehoroscope5e1a15f0 98e5 11e3 80ac 63a8ba7f...,Lifestyle_leisure
2380,dario franchitti interview indianapolis 500 ca...,Lifestyle_leisure
2381,flower power oils for loss and the stages of g...,Lifestyle_leisure


In [8]:
data = data[['TITLE','CATEGORY']]

In [9]:
#Merging both datasets
temp=[data,data1]
news_data = pd.concat(temp)
#Printing all different types of categories
news_data.CATEGORY.unique()

array(['b', 't', 'e', 'm', 'Nature_Environment', 'Politics',
       'Science_Technology', 'Economy-Business_Finance', 'Health',
       'Lifestyle_leisure'], dtype=object)

In [10]:
news_data.groupby('CATEGORY').describe()

TITLE
CATEGORY                                                                          
Economy-Business_Finance count                                                 447
                         unique                                                443
                         top                                                 share
                         freq                                                    2
Health                   count                                                 185
                         unique                                                173
                         top             Dangers of exposure to white light 21213 
                         freq                                                    3
Lifestyle_leisure        count                                                 294
                         unique                                                293
                         top     King Features Syndicate and Changing Times Vit...
                         freq                                                    2
Nature_Environment       count                                                 372
                         unique                                                351
                         top                                                 share
                         freq                                                    6
Politics                 count                                                 275
                         unique                                                272
                         top     Obama administration announces more than  2432...
                         freq                                                    3
Science_Technology       count                                                 655
                         unique                                                604
                         top                                                 share
                         freq                                                    8
b                        count                                              115967
                         unique                                             111903
                         top                                   Posted by Imaduddin
                         freq                                                   52
e                        count                                              152469
                         unique                                             146952
                         top     The article requested cannot be found! Please ...
                         freq                                                  130
m                        count                                               45639
                         unique                                              43719
                         top                                        Share this on:
                         freq                                                    9
t                        count                                              108344
                         unique                                             104733
                         top                                         Business Wire
                         freq                                                   29

In [11]:
#converting category column into numeric target NUM_CATEGORY column
news_data['NUM_CATEGORY']=news_data.CATEGORY.map({'b':0,'e':1,'m':2,'t':3,'Economy-Business_Finance':0,'Health':2,'Lifestyle_leisure':2,'Nature_Environment':4,'Politics':0,'Science_Technology':3})
news_data.head()


,TITLE,CATEGORY,NUM_CATEGORY
0,"Fed official says weak data caused by weather,...",b,0
1,Fed's Charles Plosser sees high bar for change...,b,0
2,US open: Stocks fall after Fed official hints ...,b,0
3,"Fed risks falling 'behind the curve', Charles ...",b,0
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b,0


In [12]:
#Splitting dataset into 60% training set and 40% test set
x_train, x_test, y_train, y_test = train_test_split(news_data.TITLE, news_data.NUM_CATEGORY, random_state=50)

In [13]:
#Here we convert our dataset into a Bag Of Word model using a Bigram model
start = time.clock()

vect = CountVectorizer(ngram_range=(2,2))
#converting traning features into numeric vector
X_train = vect.fit_transform(x_train)
#converting training labels into numeric vector
X_test = vect.transform(x_test)

print (time.clock()-start)



19.545383


In [14]:
#Training and Predicting the data

start = time.clock()

mnb = MultinomialNB(alpha =0.2)

mnb.fit(X_train,y_train)

result= mnb.predict(X_test)

print (time.clock()-start)

0.47650799999999904


In [15]:
#Printing accuracy of the our model
accuracy_score(result,y_test)


0.93161394849381129

In [16]:
#This function return the class of the input news
def predict_news(news):
    test = vect.transform(news)
    pred= mnb.predict(test)
    if pred  == 0:
         return 'Business or Politics'
    elif pred == 1:
        return 'Entertainment'
    elif pred == 2:
        return 'Health or Lifestyle'
    elif pred == 3:
        return 'Science and Technology'
    else:
        return 'Environment'
    
    
#'Lifestyle_leisure':4,'Nature_Environment':5,'Politics':6,

In [17]:
##Copy and paste the news headline in 'x'
x=["Nifty IT index down nearly 3% on Infosys weak guidance"]
r = predict_news(x)
print (r)

Business or Politics


In [20]:
#Printing the confusion matrix of our prediction
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, result)

array([[26317,   790,   375,  1601,     2],
       [  369, 36920,   175,   515,     0],
       [  381,   431, 10517,   319,     0],
       [ 1300,   688,   229, 25140,     0],
       [   25,    28,     5,    27,     8]])

array([[26317,   790,   375,  1601,     2],
       [  369, 36920,   175,   515,     0],
       [  381,   431, 10517,   319,     0],
       [ 1300,   688,   229, 25140,     0],
       [   25,    28,     5,    27,     8]])